# Rescale and edit training images

### Fetch Dataset from Google Drive

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! unzip /content/drive/MyDrive/FYP_GAN/training.zip -d /content/training

### Clone StyleGAN2-ADA repository

In [ ]:
! git clone https://github.com/NVlabs/stylegan2-ada.git

### Function to turn transparent pixels into white

In [ ]:
from PIL import Image

In [ ]:
# Input image has to be a PIL object
def whitenbg(img):
  pixels = img.load()
  img.convert("RGBA")
  if img.mode == "RGBA":
    for j in range(img.size[1]): # rows
      for i in range(img.size[0]): # columns
      # Check if pixel is transparent
        if pixels[i,j][3] < 255:
          # Replace with white if transparent
          pixels[i,j] = (255, 255, 255, 255)
  return img

### Function to resize images with respect to ratio

In [ ]:
# Image has to be a PIL object
def resize_img(img, width, height):

  # Keep width and height ratios
  width_ratio = width/img.width
  height_ratio = height/img.height

  # If the current height ratio is greater than the width ratio 
  if height_ratio > width_ratio:
    new_width = width # Assign desired width to new_width
    new_height = round(width_ratio * img.height) # Calculate new height with respect to width ratio
  else:
    new_height = height # Assign desired height to new_height
    new_width = round(height_ratio * img.width) # Calculate new width with respect to height ratio

  # Resize image using new sizes  
  new_img = img.resize((new_width, new_height), Image.ANTIALIAS)

  # Create new blank image with white background
  resized_image = Image.new('RGBA', (width, height), "WHITE")

  # Calculate image offset using by substracting the new sizes from the desired sizes
  image_offset = (round((width - new_width)/2), round((height - new_height)/2))
   
  # Paste the new image along with its offset onto the blank canvas
  resized_image.paste(new_img, image_offset)

  return resized_image.convert('RGB')

### Apply defined functions on dataset


In [ ]:
! mkdir /content/resized

In [ ]:
import os 

dir = "/content/training"
count = 1
for img in os.scandir(dir):
  im = Image.open(img.path)
  im = whitenbg(im)
  new_im = resize_img(im, 256, 256)
  new_im.save("/content/resized/img{}.jpg".format(count))
  print("changed ", count, " images")
  count +=1
  

### Add dataset to Google Drive and Download

In [ ]:
!zip -r /content/resized.zip /content/resized/

In [ ]:
! cp /content/resized.zip /content/drive/MyDrive/

In [ ]:
! ls /content/resized

In [ ]:
from google.colab import files
files.download("/content/resized.zip")

### Transform data into tfrecords

In [ ]:
!mkdir /content/tf_data

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
!python /content/stylegan2-ada/dataset_tool.py create_from_images /content/tf_data/ /content/resized

In [ ]:
!zip -r /content/tf_data.zip /content/tf_data